<a href="https://colab.research.google.com/github/Stalin-alt/Summer-Analytics-2025/blob/main/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy geopy

In [ ]:

import pandas as pd
import numpy as np

df = pd.read_csv("dataset.csv")

base_price = 10

alpha, beta, gamma, delta = 0.5, 0.3, 0.2, 0.5
epsilon = {'car': 1.0, 'bike': 0.7, 'truck': 1.3}
lambda_ = 0.8
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map({
    'low': 0,
    'moderate': 0.5,
    'high': 1
})

df['VehicleWeight'] = df['VehicleType'].map(epsilon)
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['QueueNorm'] = df['QueueLength'] / df['QueueLength'].max()
df['TrafficNorm'] = df['TrafficConditionNearby'] / df['TrafficConditionNearby'].max()

df['Demand'] = (
    alpha * df['OccupancyRate'] +
    beta * df['QueueNorm'] -
    gamma * df['TrafficNorm'] +
    delta * df['IsSpecialDay'] +
    df['VehicleWeight']
)

df['DemandNorm'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())
df['Model2_Price'] = base_price * (1 + lambda_ * df['DemandNorm'])
df['Model2_Price'] = df['Model2_Price'].clip(lower=0.5 * base_price, upper=2 * base_price)

print("\nMODEL 2 OUTPUT:")
print(df[['ID', 'Demand', 'Model2_Price']].head(400))




MODEL 2 OUTPUT:
      ID    Demand  Model2_Price
0      0  1.072860     12.025342
1      1  1.075459     12.037802
2      2  1.109324     12.200119
3      3  1.132721     12.312262
4      4  0.869983     11.052939
..   ...       ...           ...
395  395  1.534974     14.240288
396  396  1.069393     12.008728
397  397  1.089393     12.104590
398  398  1.079792     12.058570
399  399  0.825789     10.841113

[400 rows x 3 columns]


In [ ]:
# MODEL 2 SUMMARY:

# Demand Function:
# Demand = α * OccupancyRate + β * QueueNorm - γ * TrafficNorm + δ * IsSpecialDay + VehicleWeight
#          where α, β, γ, δ are weights controlling feature influence.

# Price = base_price * (1 + λ * NormalizedDemand)
# Normalized demand ensures all demand values lie between 0 and 1 before pricing.

# Assumptions:
# - More occupancy, more queue, and special days → higher demand.
# - Heavy traffic → lower demand (inverse relation).
# - Bigger vehicles (like trucks) demand more space → higher price weight.
# - Demand is scaled before being used to calculate price, keeping prices within realistic bounds.

# How price changes:
# - Price increases smoothly as occupancy or queues increase.
# - Bike owners are charged less, truck owners more, based on weights.
# - Model allows for flexible adjustment of each factor's impact using alpha, beta, gamma, delta.

# This is a smarter model than Model 1 — it looks at more real-world factors (queue, vehicle type, etc.).



In [ ]:
!pip install bokeh



In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

p2 = figure(title="Model 2: Price vs Normalized Demand",
            x_axis_label='Normalized Demand', y_axis_label='Price')

p2.circle(df['DemandNorm'], df['Model2_Price'], size=8, color="green", alpha=0.6)

show(p2)


PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           156                                     
    FileSystem…            0             0        18368    output          156          0                          
    FileSystem…            0             0        18368                                                            
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

KeyboardInterrupt: 

In [ ]:
import pathway as pw
import pandas as pd
from geopy.distance import geodesic


In [ ]:
class ParkingLotSchema(pw.Schema):
    ID: str
    Occupancy: int
    Capacity: int
    Latitude: float
    Longitude: float


In [ ]:
def stream_csv_rows(csv_path):
    for chunk in pd.read_csv(csv_path, chunksize=1):
        yield chunk.iloc[0].to_dict()


In [ ]:
class CompetitivePricingPipeline(pw.Pipeline):
    def __init__(self):
        super().__init__()
        self.input = pw.io.python_input(schema=ParkingLotSchema)
        self.state = []
        self.output = self.input.map(self.competitive_pricing_logic)

    def competitive_pricing_logic(self, row):
        base_price = 10
        lambda_ = 0.8
        occupancy_rate = row["Occupancy"] / row["Capacity"]
        price = base_price * (1 + lambda_ * occupancy_rate)
        price = min(max(price, 0.5 * base_price), 2 * base_price)

        # Keep a rolling state of last 100 lots for competition logic
        self.state.append({
            "ID": row["ID"],
            "Occupancy": row["Occupancy"],
            "Capacity": row["Capacity"],
            "Latitude": row["Latitude"],
            "Longitude": row["Longitude"],
            "Price": price
        })
        if len(self.state) > 100:
            self.state.pop(0)

        # Find nearby lots in state (within 3 km)
        current_coords = (row["Latitude"], row["Longitude"])
        nearby = []
        for lot in self.state:
            if lot["ID"] != row["ID"]:
                comp_coords = (lot["Latitude"], lot["Longitude"])
                if geodesic(current_coords, comp_coords).km <= 3:
                    nearby.append(lot)

        action = None
        final_price = price
        if nearby:
            nearby_prices = [lot["Price"] for lot in nearby]
            min_nearby_price = min(nearby_prices)
            max_nearby_price = max(nearby_prices)
            if row["Occupancy"] >= row["Capacity"] and price > min_nearby_price:
                final_price = min_nearby_price - 1
                action = "Suggest reroute"
            elif price < max_nearby_price:
                final_price = price + 1
                action = "Increase price"

        return {
            "ID": row["ID"],
            "Occupancy": row["Occupancy"],
            "Capacity": row["Capacity"],
            "Latitude": row["Latitude"],
            "Longitude": row["Longitude"],
            "Price": price,
            "FinalPrice": final_price,
            "Action": action
        }
